- App B: News categorisation
  - Traditional: TF-IDF + K-means
  - Neural: BERT-sentence + K-means
  - Metrics: Silhouette, Davies Bouldin, Calinski Harabasz scores + observation

In [3]:
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score
from sklearn.metrics import calinski_harabasz_score

from sentence_transformers import SentenceTransformer

import re
import spacy

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

SAMPLE_SIZE = 10000
NUM_CLUSTERS = 8
MAX_DOCS_FOR_METRICS = 3000  

def preprocess(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", "", text)      # remove URLs
    text = re.sub(r"\(cnn\)\s*--", "", text)        # remove minimal CNN header
    text = re.sub(r"[^\w\s]", " ", text)            # remove punctuation
    text = re.sub(r"\d+", " ", text)                # remove numbers
    text = re.sub(r"\s+", " ", text).strip()        # normalize whitespace

    return text

nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])  
def lemmatise(text):
    doc = nlp(text)
    return [token.lemma_ for token in doc if token.lemma_.isalpha()]

c:\Users\Jesse Tham\Desktop\School Materials\Year 4 Sem 1\EE6405 Final Project\NLPExplorationProject\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Jesse Tham\Desktop\School Materials\Year 4 Sem 1\EE6405 Final Project\NLPExplorationProject\.venv\lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [4]:
trainAll_df = pd.read_csv("../../data/Dataset2/train.csv")

print(trainAll_df.columns)
trainAll_df.head()

train_df = trainAll_df.sample(n=SAMPLE_SIZE, random_state=RANDOM_SEED).reset_index(drop=True)

Index(['id', 'article', 'highlights'], dtype='object')


In [5]:
print("Preprocessing text... this may take a few minutes.")

train_df['clean_article'] = train_df['article'].apply(preprocess)

print(train_df[['article', 'clean_article']].head())

Preprocessing text... this may take a few minutes.
                                             article  \
0  By . Mia De Graaf . Britons flocked to beaches...   
1  A couple who weighed a combined 32st were sham...   
2  Video footage shows the heart stopping moment ...   
3  Istanbul, Turkey (CNN) -- About 250 people rac...   
4  By . Daily Mail Reporter . PUBLISHED: . 12:53 ...   

                                       clean_article  
0  by mia de graaf britons flocked to beaches acr...  
1  a couple who weighed a combined st were shamed...  
2  video footage shows the heart stopping moment ...  
3  istanbul turkey about people raced across the ...  
4  by daily mail reporter published est january u...  


Traditional Model

In [6]:
# TF-IDF w stop-word removal & lemmantization
tfidf_vectorizer = TfidfVectorizer(
    tokenizer=lemmatise,
    max_features=10000,
    stop_words="english"
)

print("Fitting TF-IDF vectorizer...")
tfidf_features = tfidf_vectorizer.fit_transform(train_df['clean_article'])

print("TF-IDF feature shape:", tfidf_features.shape)

# K-means
kmeans_tfidf = KMeans(
    n_clusters=NUM_CLUSTERS,
    random_state=RANDOM_SEED,
    n_init=10
)

print("Fitting KMeans on TF-IDF features...")
kmeans_tfidf.fit(tfidf_features)

tfidf_cluster_labels = kmeans_tfidf.labels_

Fitting TF-IDF vectorizer...


c:\Users\Jesse Tham\Desktop\School Materials\Year 4 Sem 1\EE6405 Final Project\NLPExplorationProject\.venv\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\Jesse Tham\Desktop\School Materials\Year 4 Sem 1\EE6405 Final Project\NLPExplorationProject\.venv\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['I', 'far', 'make'] not in stop_words.
  warnings.warn(


TF-IDF feature shape: (10000, 10000)
Fitting KMeans on TF-IDF features...


In [ ]:
feature_names = tfidf_vectorizer.get_feature_names_out()

def get_top_words_per_cluster(kmeans_model, feature_names, top_n=15):
    cluster_keywords = []
    for cluster_id in range(kmeans_model.n_clusters):
        center = kmeans_model.cluster_centers_[cluster_id]
        top_idx = np.argsort(center)[-top_n:][::-1]
        top_words = [feature_names[i] for i in top_idx]
        cluster_keywords.append(top_words)
    return cluster_keywords

cluster_keywords_tfidf = get_top_words_per_cluster(kmeans_tfidf, feature_names, top_n=15)

for i, words in enumerate(cluster_keywords_tfidf):
    print(f"\n=== TF-IDF Cluster {i} ===")
    print(", ".join(words))


=== TF-IDF Cluster 0 ===
s, say, u, al, military, attack, government, security, syria, iraq, official, country, force, isis, president

=== TF-IDF Cluster 1 ===
s, obama, say, president, party, mr, election, tax, government, vote, I, republican, minister, house, cameron

=== TF-IDF Cluster 2 ===
say, s, court, child, I, year, school, family, mr, sentence, tell, charge, police, prison, case

=== TF-IDF Cluster 3 ===
s, league, player, club, game, cup, season, goal, team, I, win, play, score, manchester, match

=== TF-IDF Cluster 4 ===
police, say, officer, s, arrest, man, car, I, year, report, shoot, suspect, tell, old, charge

=== TF-IDF Cluster 5 ===
patient, say, health, hospital, s, disease, doctor, study, cancer, dr, cent, people, drug, woman, treatment

=== TF-IDF Cluster 6 ===
s, say, I, year, new, make, use, people, t, world, million, time, water, like, company

=== TF-IDF Cluster 7 ===
I, s, say, t, m, just, year, think, like, know, time, want, feel, make, tell


In [11]:
# Subsample for metrics
num_samples_for_metrics = min(MAX_DOCS_FOR_METRICS, tfidf_features.shape[0])
eval_indices_tfidf = np.random.choice(
    tfidf_features.shape[0],
    size=num_samples_for_metrics,
    replace=False
)

tfidf_eval = tfidf_features[eval_indices_tfidf].toarray()
tfidf_labels_eval = tfidf_cluster_labels[eval_indices_tfidf]

silhouette_tfidf = silhouette_score(tfidf_eval, tfidf_labels_eval)
davies_bouldin_tfidf = davies_bouldin_score(tfidf_eval, tfidf_labels_eval)
calinski_harabasz_tfidf = calinski_harabasz_score(tfidf_eval, tfidf_labels_eval)

print("TF-IDF + KMeans metrics:")
print("  Silhouette score       :", silhouette_tfidf)
print("  Davies-Bouldin index   :", davies_bouldin_tfidf)
print("  Calinski-Harabasz index:", calinski_harabasz_tfidf)

TF-IDF + KMeans metrics:
  Silhouette score       : 0.005059910155776732
  Davies-Bouldin index   : 9.372211147001494
  Calinski-Harabasz index: 11.56267222842952


Neural Model   

In [9]:
# Sentence-BERT
sbert_model_name = "all-MiniLM-L6-v2" 
sbert_model = SentenceTransformer(sbert_model_name)

print(f"Encoding articles with Sentence-BERT model: {sbert_model_name}...")
sbert_embeddings = sbert_model.encode(
    train_df['clean_article'],
    batch_size=32,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True, 
)

print("Sentence-BERT embedding shape:", sbert_embeddings.shape)

# K-means
kmeans_sbert = KMeans(
    n_clusters=NUM_CLUSTERS,
    random_state=RANDOM_SEED,
    n_init=10
)

print("Fitting KMeans on Sentence-BERT embeddings...")
kmeans_sbert.fit(sbert_embeddings)

sbert_cluster_labels = kmeans_sbert.labels_

c:\Users\Jesse Tham\Desktop\School Materials\Year 4 Sem 1\EE6405 Final Project\NLPExplorationProject\.venv\lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Encoding articles with Sentence-BERT model: all-MiniLM-L6-v2...


Batches: 100%|██████████| 313/313 [11:50<00:00,  2.27s/it]


Sentence-BERT embedding shape: (10000, 384)
Fitting KMeans on Sentence-BERT embeddings...


In [10]:
def get_sbert_cluster_keywords(df, labels, text_col="clean_article", top_n=15):
    cluster_keywords = []
    for c in range(NUM_CLUSTERS):
        cluster_texts = df[text_col][labels == c]
        if len(cluster_texts) == 0:
            cluster_keywords.append([])
            continue
        
        vec = TfidfVectorizer(max_features=5000, stop_words="english")
        X = vec.fit_transform(cluster_texts)
        feats = vec.get_feature_names_out()

        mean_scores = X.mean(axis=0).A1
        top_idx = np.argsort(mean_scores)[-top_n:][::-1]
        top_words = [feats[i] for i in top_idx]
        cluster_keywords.append(top_words)
    return cluster_keywords

cluster_keywords_sbert = get_sbert_cluster_keywords(
    train_df,
    sbert_cluster_labels,
    text_col="clean_article" 
)

for i, words in enumerate(cluster_keywords_sbert):
    print(f"\n=== SBERT Cluster {i} ===")
    print(", ".join(words))


=== SBERT Cluster 0 ===
said, people, police, mr, hospital, family, year, told, just, car, new, home, died, water, old

=== SBERT Cluster 1 ===
said, mr, people, year, cent, london, new, police, million, government, uk, bbc, cameron, years, tax

=== SBERT Cluster 2 ===
said, al, government, people, police, military, president, syria, security, iraq, cnn, north, country, pakistan, state

=== SBERT Cluster 3 ===
said, women, year, just, like, new, time, family, ms, says, people, years, miss, old, mother

=== SBERT Cluster 4 ===
said, obama, president, new, house, state, people, cnn, year, romney, republican, government, white, time, told

=== SBERT Cluster 5 ===
united, league, club, world, cup, season, team, england, year, said, chelsea, city, game, players, manchester

=== SBERT Cluster 6 ===
said, police, court, year, mr, told, old, home, man, school, years, family, death, mother, children

=== SBERT Cluster 7 ===
said, new, people, world, year, like, years, million, just, time, says

In [12]:
num_samples_for_metrics_sbert = min(MAX_DOCS_FOR_METRICS, sbert_embeddings.shape[0])
eval_indices_sbert = np.random.choice(
    sbert_embeddings.shape[0],
    size=num_samples_for_metrics_sbert,
    replace=False
)

sbert_eval = sbert_embeddings[eval_indices_sbert]
sbert_labels_eval = sbert_cluster_labels[eval_indices_sbert]

silhouette_sbert = silhouette_score(sbert_eval, sbert_labels_eval)
davies_bouldin_sbert = davies_bouldin_score(sbert_eval, sbert_labels_eval)
calinski_harabasz_sbert = calinski_harabasz_score(sbert_eval, sbert_labels_eval)

print("Sentence-BERT + KMeans metrics:")
print("  Silhouette score       :", silhouette_sbert)
print("  Davies-Bouldin index   :", davies_bouldin_sbert)
print("  Calinski-Harabasz index:", calinski_harabasz_sbert)

Sentence-BERT + KMeans metrics:
  Silhouette score       : 0.032252975
  Davies-Bouldin index   : 4.532999471801748
  Calinski-Harabasz index: 57.72461755129285


Comparison

In [13]:
metrics_comparison_df = pd.DataFrame(
    {
        "Model": ["TF-IDF + KMeans", "Sentence-BERT + KMeans"],
        "Silhouette": [silhouette_tfidf, silhouette_sbert],
        "Davies_Bouldin": [davies_bouldin_tfidf, davies_bouldin_sbert],
        "Calinski_Harabasz": [calinski_harabasz_tfidf, calinski_harabasz_sbert],
    }
)

metrics_comparison_df

,Model,Silhouette,Davies_Bouldin,Calinski_Harabasz
0,TF-IDF + KMeans,0.005060,9.372211,11.562672
1,Sentence-BERT + KMeans,0.032253,4.532999,57.724618
